# 02 - Enriquecimento de Dados
Autora: Fernanda Baptista de Siqueira  
Curso: MBA em Tecnologia para Negócios – AI, Data Science e Big Data  
Tema: Análise de Acidentes de Trânsito em Porto Alegre (2020–2024)  
Origem DataFrame: Equipe Armazém de Dados de Mobilidade - EAMOB/CIET  
https://dadosabertos.poa.br/dataset/acidentes-de-transito-acidentes (11/05/2025)  

### 1. Importa bibliotecas e funções. Carrega dados

In [9]:
# Imports principais
import pandas as pd
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry
import os

# Imports do projeto
from utils import (
    PATH_CHUVA, PATH_CLEAN,
    ANOS, COORD, URL,
    carregar_parquets_chuva,
    resumo_df, ajustar_tipos,
    graf_temporal, salvar_parquet
)


In [ ]:
"""
from config import (
    pd, os, salvar_parquet, resumo_df,
    COORD, PATH_CLEAN,
    ANOS, URL, PATH_CHUVA
)

import openmeteo_requests
import requests_cache
from retry_requests import retry

from pathlib import Path
"""""

### 2. Configura API Open-Meteo com cache e retry em caso de erro 

In [2]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

### 3. Cria função para chamar API

*Função para coletar dados horários de precipitação de um ponto (lat/lon)
para um determinado ano, salvando em formato parquet.*

*Args:*  
* *lat (float): Latitude*
* *lon (float): Longitude*
* *nome (str): Nome da região (ex.: NORTE)*


In [3]:
def dados_chuva(lat: float, lon: float, nome: str):
    os.makedirs(PATH_CHUVA, exist_ok=True)

    for ano in ANOS:
        # regra especial para 2025
        if ano == 2025:
            end_date = f"{ano}-05-31"
        else:
            end_date = f"{ano}-12-31"

        params = {
            "latitude": lat,
            "longitude": lon,
            "start_date": f"{ano}-01-01",
            "end_date": end_date,
            "hourly": "precipitation",
            "timezone": "America/Sao_Paulo",
        }

        responses = openmeteo.weather_api(URL, params=params)
        response = responses[0]
        hourly = response.Hourly()

        chuva_hora = hourly.Variables(0).ValuesAsNumpy()

        df_chuva = pd.DataFrame({
            "data_meteo": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True).tz_convert("America/Sao_Paulo"),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True).tz_convert("America/Sao_Paulo"),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ).tz_convert("America/Sao_Paulo").tz_localize(None),
            "chuva": chuva_hora,
        })

        nome_arquivo = f"{PATH_CHUVA}{nome.upper()}_{ano}.parquet"
        salvar_parquet(df_chuva, nome_arquivo)
        resumo_df(df_chuva)

    return df_chuva


### 4. Chama API por região  
1) Região NORTE

In [4]:
lat, lon = COORD["NORTE"]
dados_chuva(lat, lon, "NORTE")

Salvo: ../dados/intermediarios/clima/NORTE_2020.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2020-01-01 00:00:00,0.00
1,2020-01-01 01:00:00,0.00
2,2020-01-01 02:00:00,0.00
3,2020-01-01 03:00:00,0.20
4,2020-01-01 04:00:00,0.70


Salvo: ../dados/intermediarios/clima/NORTE_2021.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2021-01-01 00:00:00,0.00
1,2021-01-01 01:00:00,0.00
2,2021-01-01 02:00:00,0.00
3,2021-01-01 03:00:00,0.00
4,2021-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/NORTE_2022.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


...

Salvo: ../dados/intermediarios/clima/NORTE_2024.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2024-01-01 00:00:00,0.00
1,2024-01-01 01:00:00,0.00
2,2024-01-01 02:00:00,0.00
3,2024-01-01 03:00:00,0.00
4,2024-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/NORTE_2025.parquet
Dimensões: (3624, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00
...,...,...
3619,2025-05-31 19:00:00,0.00
3620,2025-05-31 20:00:00,0.00
3621,2025-05-31 21:00:00,0.00
3622,2025-05-31 22:00:00,0.00


2) Região LESTE

In [5]:
lat, lon = COORD["LESTE"]
dados_chuva(lat, lon, "LESTE")

Salvo: ../dados/intermediarios/clima/LESTE_2020.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2020-01-01 00:00:00,0.00
1,2020-01-01 01:00:00,0.00
2,2020-01-01 02:00:00,0.00
3,2020-01-01 03:00:00,0.00
4,2020-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/LESTE_2021.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2021-01-01 00:00:00,0.00
1,2021-01-01 01:00:00,0.00
2,2021-01-01 02:00:00,0.00
3,2021-01-01 03:00:00,0.00
4,2021-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/LESTE_2022.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


...

Salvo: ../dados/intermediarios/clima/LESTE_2024.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2024-01-01 00:00:00,0.00
1,2024-01-01 01:00:00,0.00
2,2024-01-01 02:00:00,0.00
3,2024-01-01 03:00:00,0.00
4,2024-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/LESTE_2025.parquet
Dimensões: (3624, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00
...,...,...
3619,2025-05-31 19:00:00,0.00
3620,2025-05-31 20:00:00,0.00
3621,2025-05-31 21:00:00,0.00
3622,2025-05-31 22:00:00,0.00


3) Região CENTRO

In [6]:
lat, lon = COORD["CENTRO"]
dados_chuva(lat, lon, "CENTRO")

Salvo: ../dados/intermediarios/clima/CENTRO_2020.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2020-01-01 00:00:00,0.00
1,2020-01-01 01:00:00,0.00
2,2020-01-01 02:00:00,0.00
3,2020-01-01 03:00:00,0.00
4,2020-01-01 04:00:00,1.30


Salvo: ../dados/intermediarios/clima/CENTRO_2021.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2021-01-01 00:00:00,0.00
1,2021-01-01 01:00:00,0.00
2,2021-01-01 02:00:00,0.00
3,2021-01-01 03:00:00,0.00
4,2021-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/CENTRO_2022.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


...

Salvo: ../dados/intermediarios/clima/CENTRO_2024.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2024-01-01 00:00:00,0.00
1,2024-01-01 01:00:00,0.00
2,2024-01-01 02:00:00,0.00
3,2024-01-01 03:00:00,0.00
4,2024-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/CENTRO_2025.parquet
Dimensões: (3624, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00
...,...,...
3619,2025-05-31 19:00:00,0.00
3620,2025-05-31 20:00:00,0.00
3621,2025-05-31 21:00:00,0.00
3622,2025-05-31 22:00:00,0.00


4) Região SUL

In [7]:
lat, lon = COORD["SUL"]
dados_chuva(lat, lon, "SUL")

Salvo: ../dados/intermediarios/clima/SUL_2020.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2020-01-01 00:00:00,0.00
1,2020-01-01 01:00:00,0.00
2,2020-01-01 02:00:00,0.00
3,2020-01-01 03:00:00,0.00
4,2020-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/SUL_2021.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2021-01-01 00:00:00,0.00
1,2021-01-01 01:00:00,0.00
2,2021-01-01 02:00:00,0.00
3,2021-01-01 03:00:00,0.00
4,2021-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/SUL_2022.parquet
Dimensões: (8760, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


...

Salvo: ../dados/intermediarios/clima/SUL_2024.parquet
Dimensões: (8784, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2024-01-01 00:00:00,0.00
1,2024-01-01 01:00:00,0.00
2,2024-01-01 02:00:00,0.00
3,2024-01-01 03:00:00,0.00
4,2024-01-01 04:00:00,0.00


Salvo: ../dados/intermediarios/clima/SUL_2025.parquet
Dimensões: (3624, 2)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
dtype: int64


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00


,data_meteo,chuva
0,2025-01-01 00:00:00,0.00
1,2025-01-01 01:00:00,0.00
2,2025-01-01 02:00:00,0.00
3,2025-01-01 03:00:00,0.00
4,2025-01-01 04:00:00,0.00
...,...,...
3619,2025-05-31 19:00:00,0.00
3620,2025-05-31 20:00:00,0.00
3621,2025-05-31 21:00:00,0.00
3622,2025-05-31 22:00:00,0.00


### 5. Unifica Dataset
1) Concatena Datasets

In [8]:
"""
pasta = Path(PATH_CHUVA)
dfs = []

for f in pasta.iterdir():
    if f.suffix == ".parquet":
        nome = f.stem
        regiao, ano = nome.split("_") # ignora o ano

        df_tmp = pd.read_parquet(f)
        df_tmp["regiao"] = regiao.upper()
        dfs.append(df_tmp)

df_meteo = pd.concat(dfs, ignore_index=True)

# Ajusta tipo coluna para Categoria 
df_meteo["regiao"] = df_meteo["regiao"].astype("category")
"""
df_meteo = carregar_parquets_chuva(PATH_CHUVA)
resumo_df(df_meteo)



Dimensões: (189888, 3)

Tipos de dados:
data_meteo    datetime64[ns]
chuva                float32
regiao              category
dtype: object

Nulos por coluna:
data_meteo    0
chuva         0
regiao        0
dtype: int64


,data_meteo,chuva,regiao
0,2023-01-01 00:00:00,0.00,LESTE
1,2023-01-01 01:00:00,0.00,LESTE
2,2023-01-01 02:00:00,0.00,LESTE
3,2023-01-01 03:00:00,0.00,LESTE
4,2023-01-01 04:00:00,0.00,LESTE


2. Faz merge com Dataset

In [10]:
df_limpo = pd.read_parquet(f"{PATH_CLEAN}df_limpo.parquet")

df_final = pd.merge_asof(
    df_limpo.sort_values("data_hora"),
    df_meteo.sort_values("data_meteo"),
    left_on="data_hora",
    right_on="data_meteo",
    by="regiao",  # garante que só junta se for mesma região
    direction="backward",  # pega o dado horário mais próximo anterior
    tolerance=pd.Timedelta("1h")  # permite diferença de até 1 hora
)

3. Cria coluna 0/1 para indicar se choveu ou não

In [11]:
df_final["chovendo"] = (df_final["chuva"] > 0).astype(int)

4. Valida merge

In [12]:
print("Acidentes totais:", len(df_limpo))
print("Acidentes com dado de chuva:", df_final["chuva"].notna().sum())
print("Proporção:", df_final["chuva"].notna().mean())

print("\nDistribuição da flag chovendo:")
print(df_final["chovendo"].value_counts(normalize=True))

resumo_df(df_final)

Acidentes totais: 68837
Acidentes com dado de chuva: 68837
Proporção: 1.0

Distribuição da flag chovendo:
chovendo
0   0.81
1   0.19
Name: proportion, dtype: float64
Dimensões: (68837, 35)

Tipos de dados:
predial1                   Int32
queda_arr                  Int32
data              datetime64[ns]
feridos                    Int32
feridos_gr                 Int32
fatais                     Int32
auto                       Int32
taxi                       Int32
lotacao                    Int32
onibus_urb                 Int32
onibus_met                 Int32
onibus_int                 Int32
caminhao                   Int32
moto                       Int32
carroca                    Int32
bicicleta                  Int32
outro                      Int32
cont_vit                   Int32
ups                        Int32
patinete                   Int32
idacidente                 Int32
log1              string[python]
log2              string[python]
tipo_acid               category
di

,predial1,queda_arr,data,feridos,feridos_gr,fatais,auto,taxi,lotacao,onibus_urb,onibus_met,onibus_int,caminhao,moto,carroca,bicicleta,outro,cont_vit,ups,patinete,idacidente,log1,log2,tipo_acid,dia_sem,hora,noite_dia,regiao,hora_int,data_hora,total_vitimas,soma_veiculos,data_meteo,chuva,chovendo
0,2500,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669196,AV FARRAPOS,AV SAO PEDRO,ABALROAMENTO,Quarta,0 days 02:20:00,NOITE,NORTE,2,2020-01-01 02:20:00,0,2,2020-01-01 02:00:00,0.00,0
1,598,0,2020-01-01,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,5,0,669089,AV BENTO GONCALVES,<NA>,ABALROAMENTO,Quarta,0 days 03:00:00,NOITE,LESTE,3,2020-01-01 03:00:00,1,2,2020-01-01 03:00:00,0.00,0
2,0,0,2020-01-01,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,669206,R SANTA FLORA,AV DA CAVALHADA,COLISÃO,Quarta,0 days 17:15:00,DIA,SUL,17,2020-01-01 17:15:00,0,2,2020-01-01 17:00:00,0.40,1
3,399,0,2020-01-01,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,669195,R SAO FRANCISCO DE ASSIS,<NA>,EVENTUAL,Quarta,0 days 17:15:00,DIA,NORTE,17,2020-01-01 17:15:00,0,1,2020-01-01 17:00:00,5.70,1
4,400,0,2020-01-01,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,5,0,683303,AV SENADOR TARSO DUTRA,<NA>,ABALROAMENTO,Quarta,0 days 23:00:00,NOITE,LESTE,23,2020-01-01 23:00:00,1,2,2020-01-01 23:00:00,0.00,0


5. Salva Dataset final em formato parquet

In [13]:
salvar_parquet(df_final, f"{PATH_CLEAN}df_limpo_chuva.parquet")

Salvo: ../dados/intermediarios/df_limpo_chuva.parquet


--------------------------------------
## Referências Fundamentais

- Hyndman, R. J.; Athanasopoulos, G. (2021). *Forecasting: Principles and Practice*. OTexts:contentReference[oaicite:2]{index=2}.  
- Scopel, R.; Rodrigues, R. D. (2021). *Data Science e Inteligência Artificial para melhorar a eficiência nas finanças*. PUCRS:contentReference[oaicite:3]{index=3}.  
- Box, G. E. P.; Jenkins, G. M.; Reinsel, G. C.; Ljung, G. M. (2015). *Time Series Analysis: Forecasting and Control*. Wiley.  
- Zabala, F. J. (2019). *Modelagem Preditiva*. PUCRS:contentReference[oaicite:4]{index=4}.  
- Faceli, K. et al. (2021). *Inteligência Artificial: Uma Abordagem de Aprendizado de Máquina*. LTC.  
